In [ ]:
# MNIST and Deep learning CNN
import torch
import torchvision.datasets as datasets # 이미지 관련 파이토치 라이브러리
import torchvision.transforms as transforms # 전처리 기능 제공하는 라이브러리
import torch.nn as nn # torch 내의 세부적인 기능 불러오기 (신경망 기술, 손실함수 등)
import torch.nn.functional as F # 신경망 기술
import torch.optim as optim # 최적화 문제
import matplotlib.pyplot as plt # 시각화 도구

In [ ]:
# CPU/GPU
# GPU 쓰려구. CNN은 GPU에서 효율적인 연산 가능
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'{device} is available.')

cuda:0 is available.


In [ ]:
# parameters
learning_rate = 0.01
training_epochs = 5
batch_size = 100

In [ ]:
# MNIST dataset
mnist_train = datasets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

In [ ]:
# 일반적으로 데이터셋은 이미지와 라벨이 동시에 들어있는 튜플(tuple) 형태 (이미지, 라벨)
# mnist_train[0]은 학습 데이터의 첫 번째 데이터로 이미지 한 장과 라벨 숫자 하나가 저장되어 있음
# 즉, mnist_train[0][0]은 이미지이며 mnist_train[0][1]은 라벨임
print(mnist_train.classes)

In [ ]:
print(mnist_train[10])

In [ ]:
# DataLoader는 데이터를 미니 배치 형태로 만들어 줌
# 따라서 배치 사이즈 및 셔플 여부 등을 선택할 수 있음
# 여기서 배치 사이즈는 한번 학습을 할 때마다 이미지가 100개씩 들어감을 의미
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
data_iter = iter(train_loader)
images, labels = data_iter.next()

print(images.size()) # 배치크기 X 채널 수 X 너비 X 높이 
print(labels) 

In [ ]:
# CNN Model
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = nn.Sequential(
            )
        
        # L3 ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 7, 7, 128)
        #    Pool      ->(?, 4, 4, 128)
        self.layer3 = nn.Sequential(
            )

        # L4 FC 4x4x128 inputs -> 625 outputs
        self.fc1 = nn.Linear(4 * 4 * 128, 625, bias=True)
        
        self.layer4 = nn.Sequential(
            self.fc1,
            nn.ReLU(),
            nn.Dropout(0.5))
        
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = nn.Linear(625, 10, bias=True)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1) # Flatten them for FC
        # 일렬로 핀 개수가 배치 수만큼 있어야하니까 out.size(0)

        out = self.layer4(out)
        out = self.fc2(out)
        # CrossEntropyLoss는 softmax 계산까지 포함되어 있으므로 모델의 마지막 output node에 별도의 활성화 함수를 사용하지 않아도 됨

        return out

In [ ]:
# CNN 모델 선언
# GPU 연산을 할 수 있도록 반드시 to(device)를 써줘야함
model = CNN().to(device)

In [ ]:
# CNN Model
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = nn.Sequential(
            )
        
        # L3 ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 7, 7, 128)
        #    Pool      ->(?, 4, 4, 128)
        self.layer3 = nn.Sequential(
            )

        # L4 FC 4x4x128 inputs -> 625 outputs
        self.fc1 = nn.Linear(4 * 4 * 128, 625, bias=True)
        
        self.layer4 = nn.Sequential(
            self.fc1,
            nn.ReLU(),
            nn.Dropout(0.5))
        
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = nn.Linear(625, , bias=True)
        
    def forward(self, x):
        out = self.layer1(x)
        
        
        out = out.view(out.size(0), -1) # Flatten them for FC
        # 일렬로 핀 개수가 배치 수만큼 있어야하니까 out.size(0)

        out = self.layer4(out)
        out = self.fc2(out)
        # CrossEntropyLoss는 softmax 계산까지 포함되어 있으므로 
        # 모델의 마지막 output node에 별도의 활성화 함수를 사용하지 않아도 됨

        return out

In [ ]:
print(model)

In [ ]:
# loss 함수, 옵티마이저 정의
criterion = nn.CrossEntropyLoss().to(device) # 다중 분류 문제라서 사용. softmax 계산까지 포함됨
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # 업데이트 되어야하는 파라미터 첫번째 인자로 넘겨주기

In [ ]:
len(train_loader) # 학습 이미지가 60,000장이고 배치 사이즈가 100장이므로 총 배치의 개수는 600개

In [ ]:
loss_ = [] # 그래프를 그리기 위한 loss 저장용 리스트 
total_batch = len(train_loader) # 총 배치 수
model.train()    # set the model to train mode (dropout=True)
print('!학습 시작!')
for epoch in range(training_epochs): # 5번 학습 진행
    avg_loss = 0

    for X, Y in train_loader:
        X = X.to(device) # GPU용 데이터(텐서)로 바꿔주기
        Y = Y.to(device)

        optimizer.zero_grad() # 옵티마이저 초기화
        output = model(X) # 예측값 산출
        loss = criterion(output, Y) # 손실함수 계산
        loss.backward() # 손실함수 기준으로 역전파
        optimizer.step() # 가중치 최적화

        avg_loss += loss / total_batch

    print('[Epoch: {:>2}] loss = {:>.9}'.format(epoch + 1, avg_loss))
    loss_.append(avg_loss.item()) # loss는 tensor 형태이기 때문에 item() 함수를 통해 숫자만 가져옴

print('!학습 끝!')

In [ ]:
loss_

In [ ]:
plt.plot(loss_)
plt.title("Training Loss")
plt.xlabel("epoch")
plt.show()

In [ ]:
# Test model and check accuracy
with torch.no_grad():
    model.eval() # 모델을 평가모드로 설정 (dropout=False)

    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    # prediction에는 10개의 라벨에 대한 벡터 값들이 나타나있으므로 그 중 가장 큰 값 선택해야함
    # torch.argmax(prediction, 1)의 결과는 최댓값의 위치를 나타냄
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())